In [ ]:
!pip install dimod
!pip install tqdm
!pip install dwave-neal
!pip install networkx
!pip install matplotlib
!pip install pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 73.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import random
import json
import time
import matplotlib.pyplot as plt
import networkx as nx
from tqdm.notebook import tqdm
import dimod
import neal
import gc


In [ ]:
USE_SYNERGY = True
USE_PRECEDENCE = True
USE_QUBO_SOLVER = True
SPOOF_SEVERITY = 0.1
LOCAL_VIEW_RADIUS = 2
NUM_READS = 50

FLOORS = 4
X_ROOMS = 10
Y_ROOMS = 10
NUM_AGENTS = 100
TOTAL_ROUNDS = 10
CURING_BUDGET = int(NUM_AGENTS * 0.2)
BLOCK_FRACTION = 0.03
PROB_ATTACK_ROUND = 0.85
SPOOF_FRAC_RANGE = (0.05, 0.2)
SPOOF_SEVERITY_RANGE = (SPOOF_SEVERITY, SPOOF_SEVERITY)


In [ ]:
hospital_graph = nx.Graph()
room_ids = {}

for z in range(FLOORS):
    for y in range(Y_ROOMS):
        for x in range(X_ROOMS):
            node_id = f"R_{z}_{y}_{x}"
            hospital_graph.add_node(
                node_id,
                type="room",
                x=x * 10, y=y * 10, z=z * 10,
                urgency=round(random.uniform(0.1, 1.0), 2),
                structural_integrity=round(random.uniform(0.6, 1.0), 2),
                occupants=random.randint(0, 6),
                defense_level=round(random.uniform(0.0, 0.15), 2)
            )
            room_ids[(x, y, z)] = node_id

for z in range(FLOORS):
    for y in range(Y_ROOMS):
        for x in range(X_ROOMS - 1):
            hospital_graph.add_edge(room_ids[(x, y, z)], room_ids[(x + 1, y, z)], edge_type="hallway", status="open", distance=10)
        for x in range(X_ROOMS):
            if y < Y_ROOMS - 1:
                hospital_graph.add_edge(room_ids[(x, y, z)], room_ids[(x, y + 1, z)], edge_type="hallway", status="open", distance=10)
    if z < FLOORS - 1:
        hospital_graph.add_edge(room_ids[(0, 0, z)], room_ids[(0, 0, z + 1)], edge_type="elevator", status="open", distance=10)

hospital_graph.add_node("EXIT", type="exit", x=-10, y=-10, z=-10, urgency=0.0, structural_integrity=1.0, occupants=0, defense_level=1.0)
for (x, y, z) in [(0, 0, 0), (0, 1, 0), (1, 0, 0)]:
    hospital_graph.add_edge("EXIT", room_ids[(x, y, z)], edge_type="entryway", status="open", distance=5)

tasks = [n for n, d in hospital_graph.nodes(data=True) if d['type'] == 'room']
for _ in range(10):
    bait_id = f"BAIT_{random.randint(1000, 9999)}"
    hospital_graph.add_node(
        bait_id,
        type="room",
        x=random.randint(0, 100),
        y=random.randint(0, 100),
        z=random.randint(0, 100),
        urgency=0.99,
        structural_integrity=0.2,
        occupants=1,
        defense_level=0.05
    )
    hospital_graph.add_edge(bait_id, "EXIT", edge_type="hallway", status="open", distance=10)
    tasks.append(bait_id)

for _ in range(15):
    task_id = f"TEAM_{random.randint(1000, 9999)}"
    hospital_graph.add_node(
        task_id,
        type="room",
        x=random.randint(0, 100),
        y=random.randint(0, 100),
        z=random.randint(0, 100),
        urgency=round(random.uniform(0.4, 1.0), 2),
        structural_integrity=round(random.uniform(0.6, 0.95), 2),
        occupants=random.randint(3, 6),
        defense_level=round(random.uniform(0.05, 0.15), 2)
    )
    hospital_graph.add_edge(task_id, "EXIT", edge_type="hallway", status="open", distance=10)
    tasks.append(task_id)


degree_centrality = nx.degree_centrality(hospital_graph)
for node in hospital_graph.nodes:
    hospital_graph.nodes[node]['centrality'] = degree_centrality.get(node, 0.0)


In [ ]:
def init_robots(num_agents, tasks):
    robots = {}
    for i in range(num_agents):
        initial_trust = round(random.uniform(0.5, 0.95), 2)
        skills = [round(random.uniform(0.5, 1.0), 2) for _ in range(len(tasks))]
        role = 'cleaner' if i < int(num_agents * 0.2) else 'worker'
        robots[i] = {
            'trust': initial_trust,
            'trust_history': [],
            'compromised': False,
            'success_count': 0,
            'fail_count': 0,
            'assigned_room': "EXIT",
            'spoofed_room': "EXIT",
            'skills': skills,
            'role': role,
            'initial_trust': initial_trust,
        }
    return robots

all_shortest_paths = dict(nx.all_pairs_dijkstra_path_length(hospital_graph, weight='distance'))

def get_distance(start_room, target_room):
    return all_shortest_paths.get(start_room, {}).get(target_room, 999)


def update_dynamic_blockages(graph):
    edges = list(graph.edges(data=True))
    num_to_toggle = max(1, int(BLOCK_FRACTION * len(edges)))
    to_toggle = random.sample(edges, num_to_toggle)
    for u, v, data in to_toggle:
        if data.get('status') == 'open':
            data['status'] = 'blocked'
            data['distance'] = 999
        else:
            data['status'] = 'open'
            data['distance'] = 24 if data['edge_type'] == 'hallway' else 15


In [ ]:
def advanced_spoofing_attack(robots, tasks, graph):
    if random.random() > PROB_ATTACK_ROUND:
        return

    spoof_fraction = random.uniform(*SPOOF_FRAC_RANGE)
    spoof_severity = random.uniform(*SPOOF_SEVERITY_RANGE)
    spread_probability = random.uniform(0.01, 0.05)

    num_to_spoof = max(1, int(spoof_fraction * len(robots)))
    sorted_robots = sorted(robots.items(), key=lambda item: -item[1]['trust'])
    targeted_ids = [rid for rid, _ in sorted_robots[:num_to_spoof]]

    for rid in targeted_ids:
        if robots[rid]['role'] != 'cleaner':
            robots[rid]['compromised'] = True

    spoofed_rooms = {rid: r['spoofed_room'] for rid, r in robots.items()}

    distance_cache = {}
    for rid, r in robots.items():
        if not r['compromised']:
            continue
        distance_cache[rid] = {}
        for other_id, other_room in spoofed_rooms.items():
            if rid == other_id:
                continue
            dist = get_distance(r['spoofed_room'], other_room)
            distance_cache[rid][other_id] = dist

    for rid, r in robots.items():
        if not r['compromised']:
            continue
        if rid not in distance_cache:
            continue
        for other_id, dist in distance_cache[rid].items():
            other_robot = robots[other_id]
            spread_chance = 0.3 * spread_probability if other_robot['role'] == 'cleaner' else spread_probability
            if dist < 30 and random.random() < spread_chance:
                robots[other_id]['compromised'] = True

    for rid, r in robots.items():
        if r['compromised']:
            decoy = random.choice(tasks)
            while decoy == r['assigned_room']:
                decoy = random.choice(tasks)
            r['spoofed_room'] = decoy
            r['trust'] = max(0.05, r['trust'] - spoof_severity)



def apply_environmental_noise(robots, graph):
    for r in robots.values():
        curr_room = r['spoofed_room']
        integrity = graph.nodes[curr_room].get('structural_integrity', 0.9)
        if integrity < 0.7:
            noise = np.random.normal(loc=0.0, scale=(1.0 - integrity) * 0.03)
            r['trust'] = min(0.95, max(0.05, r['trust'] + noise))

def decentralized_message_passing(robots, graph):
    agent_rooms = {rid: r['assigned_room'] for rid, r in robots.items()}

    visible_pairs = {}
    for rid1, room1 in agent_rooms.items():
        visible_pairs[rid1] = {}
        for rid2, room2 in agent_rooms.items():
            if rid1 == rid2:
                continue
            dist = get_distance(room1, room2)
            if dist <= LOCAL_VIEW_RADIUS * 10:
                visible_pairs[rid1][rid2] = dist

    for rid, r in robots.items():
        neighbors = [(other_id, robots[other_id]) for other_id in visible_pairs.get(rid, {})]
        if neighbors:
            top_neighbor = max(neighbors, key=lambda x: x[1]['trust'])[1]
            r['assigned_room'] = top_neighbor['assigned_room']


def reset_spoofed_rooms(robots):
    for r in robots.values():
        r['spoofed_room'] = r['assigned_room']


In [ ]:
def define_precedence_chains(tasks):
    task_sequence = sorted(tasks)
    precedence_map = {}
    for i in range(1, len(task_sequence)):
        if random.random() < 0.6:
            prev = task_sequence[i - 1]
            curr = task_sequence[i]
            if curr not in precedence_map:
                precedence_map[curr] = []
            precedence_map[curr].append(prev)
    return precedence_map


In [ ]:
def greedy_assignment(robots, tasks, graph, precedence_map=None):
    task_assignments = {task: [] for task in tasks}
    for rid, r in robots.items():
        best_task = None
        best_score = -np.inf
        for t in tasks:
            if get_distance(r['assigned_room'], t) > LOCAL_VIEW_RADIUS * 10:
                continue

            if precedence_map and t in precedence_map:
                if any(dep not in [r['assigned_room'] for r in robots.values()] for dep in precedence_map[t]):
                    continue

            urgency = graph.nodes[t].get('urgency', 0.2)
            trust = r['trust']
            integrity = graph.nodes[t].get('structural_integrity', 0.9)
            occupants = graph.nodes[t].get('occupants', 1)
            defense = graph.nodes[t].get('defense_level', 0.0)
            skill = r['skills'][list(tasks).index(t) % len(r['skills'])]

            score = trust * urgency * integrity * occupants * skill - defense
            if score > best_score:
                best_score = score
                best_task = t

        if best_task:
            r['assigned_room'] = best_task
            task_assignments[best_task].append(rid)
    return task_assignments


In [ ]:
def precompute_static_costs(robots, tasks, graph):
    static_terms = {}
    quad = {}
    agent_ids = list(robots.keys())

    for agent_id, robot in robots.items():
        for t_idx, task in enumerate(tasks):
            data = graph.nodes[task]
            skill = robot['skills'][t_idx % len(robot['skills'])]
            base = data.get('structural_integrity', 0.9) * data.get('occupants', 1) * skill
            static_terms[(agent_id, task)] = base - data.get('defense_level', 0.0)

    if USE_SYNERGY:
        for task in tasks:
            occ = graph.nodes[task].get('occupants', 1)
            synergy = -0.5 * occ if occ >= 2 else 0
            for i in range(len(agent_ids)):
                for j in range(i + 1, len(agent_ids)):
                    pair = ((agent_ids[i], task), (agent_ids[j], task))
                    quad[pair] = 10.0 + synergy
    else:
        for task in tasks:
            for i in range(len(agent_ids)):
                for j in range(i + 1, len(agent_ids)):
                    pair = ((agent_ids[i], task), (agent_ids[j], task))
                    quad[pair] = 10.0

    return static_terms, quad


In [ ]:
def precompute_visible_tasks(robots, tasks, graph, radius=20):
    visible = {}
    for agent_id, r in robots.items():
        nearby = []
        for task in tasks:
            dist = get_distance(r['assigned_room'], task)
            if dist <= radius:
                nearby.append((task, dist))
        visible[agent_id] = nearby
    return visible


In [ ]:
def update_dynamic_costs_np(robots, tasks, graph, static_terms, dist_penalty=0.02, comp_penalty=1.0, agent_visible_tasks=None):
    cost_dict = {}

    for agent_id, r in robots.items():
        if agent_visible_tasks:
            visible_tasks = agent_visible_tasks.get(agent_id, [])
        else:
            visible_tasks = [(t, get_distance(r['assigned_room'], t)) for t in tasks]

        for task, dist in visible_tasks:
            key = (agent_id, task)
            base = static_terms.get(key, 1.0)
            trust = r['trust']
            urgency = graph.nodes[task].get('urgency', 0.2)

            if dist > LOCAL_VIEW_RADIUS * 10:
                continue

            cost = -1.0 * trust * urgency * base + dist_penalty * dist

            if dist < 30:
                cost -= 0.05 * (30 - dist)
            elif dist > 100:
                cost += 5.0

            if r['compromised']:
                cost += comp_penalty

            cost_dict[key] = cost

    if cost_dict:
        min_cost = min(cost_dict.values())
        max_cost = max(cost_dict.values())
        range_cost = max_cost - min_cost + 1e-6

        for k in cost_dict:
            cost_dict[k] = (cost_dict[k] - min_cost) / range_cost

    return cost_dict


In [ ]:
def build_or_update_bqm(bqm, cost_dict, quad_penalties):
    if bqm is None:
        bqm = dimod.BinaryQuadraticModel('BINARY')
        for var in cost_dict:
            bqm.add_variable(var, 0.0)
        for (var1, var2), penalty in quad_penalties.items():
            bqm.add_interaction(var1, var2, penalty)
    for var, cost in cost_dict.items():
        bqm.set_linear(var, cost)
    return bqm


In [ ]:
def apply_qubo_assignments(result, robots, tasks, precedence_map):
    task_assignments = {task: [] for task in tasks}
    assigned_tasks = set()
    assigned_rooms = set(r['assigned_room'] for r in robots.values())

    for (agent_id, task), val in result.items():
        if val != 1 or task in assigned_tasks:
            continue

        deps = precedence_map.get(task, [])
        if all(dep in assigned_rooms for dep in deps):
            robots[agent_id]['assigned_room'] = task
            task_assignments[task].append(agent_id)
            assigned_tasks.add(task)
        else:
            robots[agent_id]['assigned_room'] = "EXIT"

    return task_assignments



In [ ]:
robots = init_robots(NUM_AGENTS, tasks)
precedence_map = define_precedence_chains(tasks)
static_terms, quad_penalties = precompute_static_costs(robots, tasks, hospital_graph)
bqm = None
sampler = neal.SimulatedAnnealingSampler()

trust_over_time = []
trusturgency_scores = []
recovered_count = []
spoofed_total = []
coordinated_tasks = []
solve_times = []
bait_counts = []
recovery_rates = []
qubo_scores = []
precedence_violations = []
duplicate_assignments = []
task_distances = []


In [ ]:
summaries = []
for round_num in tqdm(range(TOTAL_ROUNDS), desc="Running Simulation"):
    reset_spoofed_rooms(robots)
    update_dynamic_blockages(hospital_graph)
    apply_environmental_noise(robots, hospital_graph)
    decentralized_message_passing(robots, hospital_graph)

    t0 = time.time()
    if USE_QUBO_SOLVER:
        agent_visible_tasks = precompute_visible_tasks(robots, tasks, hospital_graph, radius=LOCAL_VIEW_RADIUS * 10)
        cost_dict = update_dynamic_costs_np(
            robots, tasks, hospital_graph, static_terms,
            agent_visible_tasks=agent_visible_tasks
        )

        bqm = build_or_update_bqm(bqm, cost_dict, quad_penalties)
        result = sampler.sample(bqm, num_reads=NUM_READS).first.sample

        qubo_scores.append(bqm.energy(result))

        task_assignments = apply_qubo_assignments(result, robots, tasks, precedence_map)
        bait_counts.append(sum(1 for r in robots.values() if r['assigned_room'].startswith("BAIT")))
    else:
        task_assignments = greedy_assignment(robots, tasks, hospital_graph, precedence_map)
        bait_counts.append(sum(1 for r in robots.values() if r['assigned_room'].startswith("BAIT")))

    solve_times.append(time.time() - t0)

    trust_vector = np.array([r['trust'] for r in robots.values()])
    spoofed_flags = np.array([r['trust'] < 0.6 for r in robots.values()])
    num_spoofed = np.sum(spoofed_flags)
    num_recovered = np.sum((trust_vector >= 0.6) & spoofed_flags)

    spoofed_total.append(num_spoofed)
    recovered_count.append(num_recovered)

    successful_tasks = sum(1 for agents in task_assignments.values() if len(agents) >= 1)
    total_tasks = len(task_assignments)
    coordinated_tasks.append(successful_tasks / (total_tasks + 1e-6))

    violations = 0
    for task, deps in precedence_map.items():
        if task in task_assignments and deps:
            assigned_rooms = set(r['assigned_room'] for r in robots.values())
            if any(dep not in assigned_rooms for dep in deps):
                violations += len(task_assignments[task])
    precedence_violations.append(violations)

    duplicate_count = sum(1 for agents in task_assignments.values() if len(agents) > 1)
    duplicate_assignments.append(duplicate_count)

    avg_dist = np.mean([
        get_distance(r['assigned_room'], r['spoofed_room'])
        for r in robots.values()
    ])
    task_distances.append(avg_dist)

    advanced_spoofing_attack(robots, tasks, hospital_graph)
    for r in robots.values():
        outcome = random.random() < hospital_graph.nodes[r['spoofed_room']].get('structural_integrity', 0.9)
        if outcome:
            r['trust'] = min(0.95, r['trust'] + 0.02)
            r['success_count'] += 1
        else:
            r['trust'] = max(0.05, r['trust'] - 0.05)
            r['fail_count'] += 1
        r['trust_history'].append(r['trust'])

    treated_rooms = set(r['spoofed_room'] for r in robots.values())
    for node, data in hospital_graph.nodes(data=True):
        if data.get('type') != 'room':
            continue
        curr = data['urgency']
        if node in treated_rooms:
            data['urgency'] = round(max(0.1, curr - 0.05), 2)
        else:
            data['urgency'] = round(min(1.0, curr + 0.02), 2)

    total_cleaner_success = sum(r['success_count'] for r in robots.values() if r['role'] == 'cleaner')
    comp = [(rid, r) for rid, r in robots.items() if r['compromised']]
    comp.sort(key=lambda x: (-x[1]['trust'],
                             -hospital_graph.nodes[x[1]['spoofed_room']]['centrality'],
                             -x[1]['success_count']))
    for rid, r in comp[:CURING_BUDGET]:
        room_data = hospital_graph.nodes[r['spoofed_room']]
        p = (
            0.1 +
            0.02 * r['success_count'] +
            room_data['defense_level'] +
            0.001 * total_cleaner_success +
            0.05 * room_data['centrality']
        )
        if random.random() < min(1.0, p * room_data['structural_integrity']):
            r['compromised'] = False
            r['trust'] = min(0.95, r['trust'] + 0.05)

    trust_over_time.append(np.mean([r['trust'] for r in robots.values()]))
    trusturgency_scores.append(
        sum(r['trust'] * hospital_graph.nodes[r['assigned_room']]['urgency'] for r in robots.values())
    )

    if (round_num + 1) % 10 == 0:
        chunk = (round_num + 1) // 10
        pd.DataFrame({
            "round": list(range(round_num - 9, round_num + 1)),
            "avg_trust": trust_over_time[-10:]
        }).to_csv(f"trust_chunk_{chunk}.csv", index=False)

        pd.DataFrame([{
            "chunk": chunk,
            "trusturgency": np.mean(trusturgency_scores[-10:]),
            "avg_spoofed": np.mean(spoofed_total[-10:]),
            "avg_recovery": np.mean(recovered_count[-10:]),
            "avg_coord_success": np.mean(coordinated_tasks[-10:]),
            "avg_violations": np.mean(precedence_violations[-10:]),
            "avg_duplicates": np.mean(duplicate_assignments[-10:]),
            "avg_distance": np.mean(task_distances[-10:]),
            "avg_solve_time": np.mean(solve_times[-10:]),
            "avg_bait": np.mean(bait_counts[-10:]),
            "qubo_score": np.mean(qubo_scores[-10:]) if qubo_scores else None
        }]).to_csv(f"summary_chunk_{chunk}.csv", index=False)

        print(f"📦 Saved chunk {chunk}/10 at round {round_num + 1}")

    if round_num % 10 == 0:
        gc.collect()


Running Simulation:   0%|          | 0/10 [00:00<?, ?it/s]

📦 Saved chunk 1/10 at round 10


In [ ]:
summary_dict = {
    "final_trust": np.mean(trust_over_time),
    "trusturgency": np.mean(trusturgency_scores),
    "avg_spoofed": np.mean(spoofed_total),
    "avg_recovery": np.mean(recovered_count),
    "avg_coord_success": np.mean(coordinated_tasks),
    "avg_violations": np.mean(precedence_violations),
    "avg_duplicates": np.mean(duplicate_assignments),
    "avg_distance": np.mean(task_distances),
    "avg_solve_time": np.mean(solve_times),
    "avg_bait": np.mean(bait_counts),
    "qubo_score": np.mean(qubo_scores) if qubo_scores else None
}


run_label = "qubo" if USE_QUBO_SOLVER else "greedy"
pd.DataFrame([summary_dict]).to_csv("single_run_summary_4_qubo.csv", index=False)

trust_df = pd.DataFrame({
    "round": list(range(TOTAL_ROUNDS)),
    "avg_trust": trust_over_time
})
trust_df.to_csv("trust_over_time_synthetic_v4_4_qubo.csv", index=False)

print("Simulation complete! Saved output for Tab 4.")



✅ Simulation complete! Saved output for Tab 4.
